<a href="https://colab.research.google.com/github/barrosm/DeepLearningExamples/blob/master/LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualização LDA

Utilizaremos a distribuição de topicos de documentos como o vetor embedding do documento.

## 1. Preprocessamento do Texto
Usamos o dataset de sinopsis de filme como nossos documentos e removemos palavras raras e palavras comuns com base na sua frequência no documento. Removemos palavras que aparecem em menos de 2 documentos ou em mais de 30% dos documentos.

In [1]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 2.8MB/s 
     |████████████████████████████████| 552kB 47.9MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=0d95c515b9846bb2e7b342838677f7cee6394e002d157eb43cd9843529de1740
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=4295dc96b621ea248006676c8a29723f5909c88b5ec8b038166d3b8a5fb013f1
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [5]:
dataframe.shape

(1843, 4)

In [4]:
dataframe.head()

,MovieID,Titles,Plots,Genres
0,1,Toy Story (1995),little boy named andy loves room playing toys...,animation
1,2,Jumanji (1995),kids play magical board game release man trap...,fantasy
2,3,Grumpier Old Men (1995),things don t change wabasha county max john f...,comedy
3,6,Heat (1995),hunters prey neil professional criminal crew h...,action
4,7,Sabrina (1995),ugly duckling having undergone remarkable chan...,romance


In [0]:
import pandas as pd
import re
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation
from gensim.models import ldamodel
from gensim.corpora.dictionary import Dictionary

# read data
dataframe = pd.read_csv('movie_plots.csv')

# remove stopwords and punctuations
def preprocess(row):
    return strip_punctuation(remove_stopwords(row.lower()))
    
dataframe['Plots'] = dataframe['Plots'].apply(preprocess)

# Convert data to required input format by LDA
texts = []
for line in dataframe.Plots:
    lowered = line.lower()
    words = re.findall(r'\w+', lowered, flags = re.UNICODE)# | re.LOCALE
    texts.append(words)
    
# Create a dictionary representation of the documents.
dictionary = Dictionary(texts)

# Filter out words that occur less than 2 documents, or more than 30% of the documents.
dictionary.filter_extremes(no_below=2, no_above=0.3)

# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(text) for text in texts]

## 2. Treinamos o modelo LDA

In [0]:
# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 50
iterations = 200
eval_every = None

# Train model
model = ldamodel.LdaModel(corpus=corpus, id2word=dictionary, chunksize=chunksize, alpha='auto', 
                          eta='auto', iterations=iterations, num_topics=num_topics, passes=passes, 
                          eval_every=eval_every)

## 3. Distribuição de topicos de documentos
Usamos o método get_document_topics que realiza a infrencia de a distribuição de topicos de um documento. Retorna uma lista de tupa (topic_id, probability)

In [7]:
# Get document topics
all_topics = model.get_document_topics(corpus, minimum_probability=0)
all_topics[2]

[(0, 0.00032733363),
 (1, 0.00051712134),
 (2, 0.00042448545),
 (3, 0.00040002432),
 (4, 0.0003790012),
 (5, 0.0004082769),
 (6, 0.99611866),
 (7, 0.0005300765),
 (8, 0.0004480381),
 (9, 0.00044694805)]

## 4. Preparando os arquivos de entrada para TensorBoard

In [0]:
from tensorboard_helper import CreateTensorboardData

In [0]:
vectors = [ [topics[1] for topics in doc_topics] for doc_topics in all_topics]
metadatos = [dataframe.Titles.values , dataframe.Genres.values]

In [16]:
CreateTensorboardData(tensor_filename="doc_lda", 
                      vectors=vectors, 
                      metadatos=metadatos,
                      colnames=["Titles","Genres"])

Arquivo com o Tensor 2D foi salvado em: doc_lda_tensor.tsv
Arquivo com o Tensor de metadatos foi salvado em: doc_lda_metadata.tsv


Estes arquivos que a gente vai baixar vão ser usados em http://projector.tensorflow.org/


Baixar arquivos de vetores : <a href="doc_lda_tensor.tsv" download="w3logo"> doc_lda_tensor.tsv </a>

Baixar arquivos com os metadatos: <a href="doc_lda_metadata.tsv" download="w3logo"> doc_lda_metadata.tsv </a>

**Visualizando usando PCA ...**

Levar os arquivos para http://projector.tensorflow.org/.

Como podemos ver, há muitos pontos que se agrupam nos cantos. Os documentos nos cantos pertencem principalmente a um único tópico (portanto, o peso grande em uma única dimensão e outras dimensões têm peso aproximadamente zero). Você pode modificar o arquivo de metadados conforme explicado abaixo para ver os pesos das dimensões juntamente com o título do filme.

Agora, vamos anexar os tópicos com maior probabilidade (topic_id, topic_probability) ao título do documento, para explorar quais tópicos os domínios ou bordas do cluster pertencem de forma dominante. Para isso, precisamos sobrescrever o arquivo de metadados conforme abaixo:

In [0]:
tensors = []
for doc_topics in all_topics:
    doc_tensor = []
    for topic in doc_topics:
        if round(topic[1], 3) > 0:
            doc_tensor.append((topic[0], float(round(topic[1], 3))))
            
    # sort topics according to highest probabilities
    doc_tensor = sorted(doc_tensor, key=lambda x: x[1], reverse=True)
    # store vectors to add in metadata file
    tensors.append(doc_tensor[:5])

In [12]:
tensors_str = [ str(" ".join([title, str(x)])) for title, x in zip(dataframe.Titles.values, tensors)]
metadatos = [tensors_str, dataframe.Genres.values]

CreateTensorboardData(tensor_filename="doc_lda", 
                      vectors=vectors, 
                      metadatos=metadatos,
                      colnames=["Titles","Genres"])

Arquivo com o Tensor 2D foi salvado em: doc_lda_tensor.tsv
Arquivo com o Tensor de metadatos foi salvado em: doc_lda_metadata.tsv


Baixar arquivos de vetores : <a href="doc_lda_tensor.tsv" download="w3logo"> doc_lda_tensor.tsv </a>

Baixar arquivos com os metadatos: <a href="doc_lda_metadata.tsv" download="w3logo"> doc_lda_metadata.tsv </a>

**Visualizando usando T-SNE ...**

Levar os arquivos para http://projector.tensorflow.org/.

In [13]:
pwd

'/content'

In [14]:
model.show_topic(topicid=0, topn=15)

[('life', 0.0058544846),
 ('new', 0.005547385),
 ('world', 0.004965163),
 ('man', 0.004545385),
 ('finds', 0.003877505),
 ('time', 0.0036138936),
 ('love', 0.0034251958),
 ('t', 0.0033728275),
 ('young', 0.0033305318),
 ('old', 0.0032266958),
 ('it', 0.0030606245),
 ('help', 0.0029499677),
 ('friends', 0.0028829742),
 ('earth', 0.0028286492),
 ('father', 0.0028233202)]

In [15]:
import pyLDAvis.gensim

viz = pyLDAvis.gensim.prepare(model, corpus, dictionary)
pyLDAvis.display(viz)